# Procura de padrões

### Autómatos finitos

In [2]:
class Automato_finito:
    """
    Implementação de um autómato finito para busca de padrões.

    Este autómato é configurado através de um alfabeto e um padrão a serem procurados em sequências. 
    A tabela de transições é construída com base no padrão, o que permite identificar os estados percorridos e as ocorrências do padrão.
    """

    def __init__(self, alfabeto, padrao):
        """
        Inicializa o autómato finito com o alfabeto e o padrão a serem procurados.

        Parâmetros:
            alphabet (str): O alfabeto utilizado tanto no padrão quanto nas sequências.
            pattern (str): O padrão que se deseja encontrar nas sequências.
        """
        self.alfabeto = alfabeto  
        self.padrao = padrao
        self.num_estados = len(padrao) + 1  
        self.tabela_transicao = {}  
        self._constr_tabela_transicao()  

    def _constr_tabela_transicao(self):
        """
        Constrói a tabela de transições do autómato com base no padrão definido.

        Para cada estado possível e para cada caractere do alfabeto, este método calcula
        a sobreposição entre o prefixo formado pelo estado atual acrescido do caractere e o padrão, ao registar o resultado correspondente na tabela de transições.
        """
        for estado in range(self.num_estados):  
            for char in self.alfabeto:  
                prefixo = self.padrao[:estado] + char  
                self.tabela_transicao[(estado, char)] = self._sobreposicao(prefixo)  

    def _sobreposicao(self, s):
        """
        Calcula o tamanho da sobreposição entre o final da string 's' e o início do padrão.

        Este método determina quantos caracteres finais de 's' coincidem com os caracteres
        iniciais do padrão, retornando o comprimento desta sobreposição.

        Parâmetros:
            s (str): A string na qual se quer identificar a sobreposição com o padrão.

        Retorna:
            int: O tamanho da sobreposição encontrado.
        """
        max_sobreposicao = min(len(s), len(self.padrao))  
        for i in range(max_sobreposicao, 0, -1):  
            if s[-i:] == self.padrao[:i]:  
                return i  
        return 0  

    def aplicar_sequencia(self, sequencia):
        """
        Aplica o autómato a uma sequência e retorna a lista dos estados percorridos.

        O autómato processa cada caractere da sequência através da tabela de transições, registrando a evolução dos estados.

        Parâmetros:
            sequence (str): A sequência na qual o autómato será aplicado.

        Retorna:
            list[int]: Uma lista com os estados visitados durante o processamento da sequência.
        """
        estado = 0  
        estados = [estado]  
        for char in sequencia:  
            estado = self.tabela_transicao.get((estado, char), 0)  
            estados.append(estado)  
        return estados  

    def encontrar_ocorrencias_padrao(self, sequencia):
        """
        Identifica todas as ocorrências do padrão na sequência fornecida.

        Ao aplicar o autómato na sequência, este método verifica em quais posições
        o autómato alcança o estado final, indicando assim a ocorrência do padrão.

        Parâmetros:
            sequence (str): A sequência na qual se irá procurar o padrão.

        Retorna:
            list[int]: Uma lista com as posições iniciais onde o padrão ocorre na sequência.
        """
        estado = 0  
        ocorrencias = []  
        for i, char in enumerate(sequencia):
            estado = self.tabela_transicao.get((estado, char), 0)  
            if estado == self.num_estados - 1:  
                ocorrencias.append(i - len(self.padrao) + 1)  
        return ocorrencias 

if __name__ == "__main__":

    alfabeto = "AC"  
    padrao = "ACA"  

    automato = Automato_finito(alfabeto, padrao)  

    print("Tabela de Transições:")  
    for key, value in automato.tabela_transicao.items(): 
        print(f"Estado {key[0]}, Caractere '{key[1]}' -> Estado {value}")  

    sequencia = "CACAACAA"  
    estados = automato.aplicar_sequencia(sequencia)  
    print("\nEstados percorridos:", estados)  

    ocorrencias = automato.encontrar_ocorrencias_padrao(sequencia)  
    print("\nOcorrências do padrão:", ocorrencias)

Tabela de Transições:
Estado 0, Caractere 'A' -> Estado 1
Estado 0, Caractere 'C' -> Estado 0
Estado 1, Caractere 'A' -> Estado 1
Estado 1, Caractere 'C' -> Estado 2
Estado 2, Caractere 'A' -> Estado 3
Estado 2, Caractere 'C' -> Estado 0
Estado 3, Caractere 'A' -> Estado 1
Estado 3, Caractere 'C' -> Estado 2

Estados percorridos: [0, 0, 1, 2, 3, 1, 2, 3, 1]

Ocorrências do padrão: [1, 4]


### Boyer Moore

In [3]:
class BoyerMoore:
    """
    Implementação do algoritmo Boyer-Moore para procura de padrões.
    O algoritmo utiliza duas regras principais de otimização: a regra do mau caráter e a regra do bom sufixo.
    """

    def __init__(self, alfabeto, padrao):
        """
        Inicializa a classe com o alfabeto e o padrão a ser buscado.
        
        Parâmetros:
        alfabeto (str): O alfabeto utilizado no texto.
        padrao (str): O padrão a ser procurado no texto.
        """
        self.alfabeto = alfabeto
        self.padrao = padrao
        self.preprocessar()  

    def preprocessar(self):
        """
        Realiza o pré-processamento utilizando as duas regras de Boyer-Moore:
        
        1. Regra do mau caráter (BCR)
        2. Regra do bom sufixo (GSR)
        """
        self.processar_bcr()
        self.processar_gsr()

    def processar_bcr(self):
        """
        Pré-processamento da regra do mau caráter.
        Calcula a última ocorrência de cada caráter do alfabeto no padrão e armazena essas informações em um dicionário (self.ocorrencias).
        """
        self.ocorrencias = {}  
        for simbolo in self.alfabeto:
            self.ocorrencias[simbolo] = -1  
        for indice in range(len(self.padrao)):
            self.ocorrencias[self.padrao[indice]] = indice  
        print(self.ocorrencias)

    def processar_gsr(self):
        """
        Pré-processamento da regra do bom sufixo.
        Calcula o deslocamento máximo que permite avançar no padrão sem comprometer a busca.
        São criadas duas listas:
          - self.f: que armazena, para cada posição, o maior sufixo que é também um prefixo do padrão.
          - self.s: que armazena o número de posições a avançar quando ocorre uma falha.
        """
        self.f = [0] * (len(self.padrao) + 1)  
        self.s = [0] * (len(self.padrao) + 1)  
        i = len(self.padrao)             
        j = len(self.padrao) + 1         
        self.f[i] = j                  
        print(self.f)
        
        while i > 0: 
            while j <= len(self.padrao) and self.padrao[i - 1] != self.padrao[j - 1]:
                if self.s[j] == 0:
                    self.s[j] = j - i  
                j = self.f[j] 
            i -= 1
            j -= 1
            self.f[i] = j  
        j = self.f[0]
        for i in range(len(self.padrao)):
            if self.s[i] == 0:
                self.s[i] = j
            if i == j:
                j = self.f[j]
        print(self.f)
        print(self.s)

    def procurar_padrao(self, texto):
        """
        Realiza a busca do padrão no texto utilizando o algoritmo Boyer-Moore.
        
        Parâmetros:
        texto (str): O texto onde o padrão será procurado.
        
        Retorna:
        list: Uma lista com os índices onde o padrão ocorre no texto.
        """
        res = []
        i = 0
        while i <= (len(texto) - len(self.padrao)):
            j = len(self.padrao) - 1  
            while j >= 0 and self.padrao[j] == texto[j + i]:
                j -= 1
            if j < 0:
                res.append(i)  
                i += self.s[0]  
            else:
                c = texto[i + j]
                i += max(self.s[j + 1], j - self.ocorrencias[c])  
        return res


def testar():
    """
    Função de teste para verificar a implementação do algoritmo Boyer-Moore.
    """
    bm = BoyerMoore("ACTG", "ACCA")
    print(bm.procurar_padrao("ATAGAACCAATGAACCATGATGAACCATGGATACCCAACCACC"))


if __name__ == "__main__":
    testar()

{'A': 3, 'C': 2, 'T': -1, 'G': -1}
[0, 0, 0, 0, 5]
[3, 4, 4, 4, 5]
[3, 3, 3, 3, 1]
[5, 13, 23, 37]


### Trees

In [4]:
class Trie:
    """
    Implementação de uma Trie para armazenamento e pesquisa de palavras.

    A estrutura Trie permite inserir palavras e realizar pesquisas de forma eficiente, 
    tanto para verificar a existência completa de palavras como para encontrar todas as palavras que partilham um prefixo comum.
    """

    def __init__(self):
        """
        Inicia a Trie como um dicionário vazio, onde cada chave representa um caractere e cada valor representa a continuação da palavra.
        """
        self.trie = {}

    def insert(self, word):
        """
        Insere uma palavra na Trie, ao adicionar um marcador especial ('$') no final para indicar o fim da palavra.

        Parametros:
            word (str): A palavra a ser inserida na Trie.
        """
        current = self.trie
        for char in word + "$":  
            if char not in current:
                current[char] = {}  
            current = current[char]

    def search(self, word):
        """
        Verifica se uma palavra completa está armazenada na Trie.

        A pesquisa percorre a estrutura da Trie caractere a caractere e verifica no final se existe o marcador especial de fim ('$').

        Parametros:
            word (str): A palavra a procurar na Trie.

        Retorna:
            bool: True se a palavra existir na Trie, False caso contrário.
        """
        current = self.trie
        for char in word:
            if char not in current:
                return False  
            current = current[char]
        return "$" in current  

    def starts_with(self, prefix):
        """
        Encontra todas as palavras na Trie que começam com o prefixo fornecido.

        Após localizar o prefixo na Trie, percorre recursivamente todos os caminhos possíveis para recolher as palavras completas.

        Parametros:
            prefix (str): Prefixo a procurar na Trie.

        Retorna:
            list: Lista de palavras que começam com o prefixo fornecido.
        """
        current = self.trie
        for char in prefix:
            if char not in current:
                return []  
            current = current[char]
        return self._collect_words(prefix, current)

    def _collect_words(self, prefix, node):
        """
        Função auxiliar que recolhe todas as palavras completas a partir de um determinado nó da Trie.

        Esta função é chamada recursivamente até encontrar todos os caminhos terminados com o marcador especial de fim ('$').

        Parametros:
            prefix (str): Prefixo acumulado até ao nó atual.
            node (dict): Subárvore da Trie correspondente ao prefixo.

        Retorna:
            list: Lista de palavras completas derivadas do prefixo fornecido.
        """
        words = []
        for char, child in node.items():
            if char == "$":
                words.append(prefix) 
            else:
                words.extend(self._collect_words(prefix + char, child)) 
        return words


class SuffixTree:
    """
    Implementação de uma Árvore de Sufixos para armazenamento e pesquisa de substrings.

    Esta estrutura permite inserir todos os sufixos de uma string para facilitar a pesquisa eficiente de substrings, 
    devolvendo as posições onde essas substrings ocorrem no texto original.
    """

    def __init__(self):
        """
        Inicializa a Árvore de Sufixos como um dicionário vazio, onde cada caminho representa um sufixo do texto.
        """
        self.tree = {}

    def insert(self, text):
        """
        Adiciona todos os sufixos da string fornecida na Árvore de Sufixos.

        Para cada sufixo, cria-se um caminho separado na árvore, que marca o fim com o símbolo especial ('$') e regista a posição inicial do sufixo.

        Parametros:
            text (str): Texto base de onde serão criados e adicionados os sufixos.
        """
        if text == "":
            current = self.tree
            if "$" not in current:
                current["$"] = {}
            if "index" not in current["$"]:
                current["$"]["index"] = []
            current["$"]["index"].append(0)
            return

        for i in range(len(text)):
            self._insert_suffix(text[i:], i)

    def _insert_suffix(self, suffix, index):
        """
        Função auxiliar que insere um sufixo específico na Árvore de Sufixos.

        Cada caractere do sufixo é adicionado sequencialmente, e ao final regista-se a posição inicial do sufixo.

        Parametros:
            suffix (str): Sufixo a ser inserido.
            index (int): Posição inicial do sufixo na string original.
        """
        current = self.tree
        for char in suffix + "$":
            if char not in current:
                current[char] = {}
            current = current[char]
            if char == "$":
                if "index" not in current:
                    current["index"] = []
                if index not in current["index"]:
                    current["index"].append(index)

    def search(self, substring):
        """
        Procura todas as posições onde uma determinada substring ocorre na Árvore de Sufixos.

        A pesquisa percorre a árvore seguindo os caracteres da substring e, caso encontrada, recolhe todas as posições associadas.

        Parâmetros:
            substring (str): Substring a procurar na Árvore de Sufixos.

        Retorna:
            list: Lista das posições iniciais onde a substring ocorre no texto original.
        """
        if substring == "":
            return []

        current = self.tree
        for char in substring:
            if char not in current:
                return []  
            current = current[char]
        return self._collect_positions(current)

    def _collect_positions(self, node):
        """
        Função auxiliar que recolhe todas as posições associadas a partir de um determinado nó da árvore.

        Esta função é chamada recursivamente para recolher todos os índices armazenados nas folhas.

        Parâmetros:
            node (dict): Nó atual da Árvore de Sufixos.

        Retorna:
            list: Lista ordenada de posições onde a substring ocorre.
        """
        positions = []
        for key, child in node.items():
            if key == "index":
                positions.extend(child)
            elif isinstance(child, dict):
                positions.extend(self._collect_positions(child))
        return sorted(positions)

if __name__ == "__main__":
    print("=== Exemplo de trie ===")
    trie = Trie()
    words = ["apple", "app", "banana", "band", "bandana"]
    for word in words:
        trie.insert(word)

    print("Procura 'app':", trie.search("app"))  
    print("Procura 'apple':", trie.search("apple")) 
    print("Procura 'apples':", trie.search("apples"))  
    print("Palavras começadas por 'ban':", trie.starts_with("ban"))  

    print("\n=== Exemplo de suffix tree ===")
    suffix_tree = SuffixTree()
    text = "banana"
    suffix_tree.insert(text)

    print("Procura 'ana':", suffix_tree.search("ana"))  
    print("Procura 'ban':", suffix_tree.search("ban"))  
    print("Procura 'xyz':", suffix_tree.search("xyz"))  

=== Exemplo de trie ===
Procura 'app': True
Procura 'apple': True
Procura 'apples': False
Palavras começadas por 'ban': ['banana', 'band', 'bandana']

=== Exemplo de suffix tree ===
Procura 'ana': [1, 3]
Procura 'ban': [0]
Procura 'xyz': []


# BWT

In [5]:

class BWT:
    def __init__(self, seq="", constroi_array_sufixos=True):
        """
        Inicializa uma instância da classe BWT.

        Cria a transformação de Burrows-Wheeler (BWT) a partir da sequência fornecida e,
        opcionalmente, constroi o array de sufixos.

        Args:
            seq (str, optional): Sequência de caracteres a processar. Por defeito, é uma string vazia.
            constroi_array_sufixos (bool, optional): Se True, também constrói o array de sufixos. Por defeito é True.
        """
        self.bwt = self.constroi_bwt(seq, constroi_array_sufixos)            

    def define_bwt(self, bw):
        """
        Define manualmente a BWT.

        Args:
            bw (str): String que representa a BWT a definir.
        """
        self.bwt = bw                                                       

    def constroi_bwt(self, texto, constroi_array_sufixos=False):
        """
        Constrói a BWT a partir do texto fornecido.

        Gera todas as rotações do texto, ordena-as e constrói a BWT a partir do último
        caractere de cada linha ordenada. Se solicitado, constrói também o array de sufixos.

        Args:
            texto (str): Texto de entrada.
            constroi_array_sufixos (bool, optional): Se True, o array de sufixos é também construído. Por defeito é False.

        Returns:
            str: A BWT correspondente à sequência de entrada.
        """
        lista = []                                          
        for i in range(len(texto)):
            lista.append(texto[i:] + texto[:i])             
        lista.sort()                                         
        resultado = ""
        for j in range(len(texto)):                        
            resultado += lista[j][-1]

        if constroi_array_sufixos:
            self.sa = []                                     
            for i in range(len(lista)):
                pos_inicial = lista[i].index("$")           
                self.sa.append(len(texto) - pos_inicial - 1)
        return resultado

    def inversa_bwt(self):
        """
        Inverte a BWT para recuperar o texto original.

        Reconstrói o texto original a partir da BWT, usando o método da tabela iterativa.

        Returns:
            str: O texto original se a inversão for bem-sucedida; caso contrário, retorna uma string vazia.
        """
        n = len(self.bwt)                                                    
        tabela = [""] * n                                                    
        for _ in range(n):
            tabela = sorted([self.bwt[i] + tabela[i] for i in range(n)])      
        for linha in tabela:                                                 
            if linha.endswith("$"):                                          
                return linha
        return ""                                                             

    def obtem_primeira_coluna(self):
        """
        Obtém a primeira coluna da matriz da BWT.

        A primeira coluna é a versão ordenada da BWT.

        Returns:
            list: Lista que representa a primeira coluna da matriz BWT.
        """
        primeira_coluna = sorted(self.bwt)                                    
        return primeira_coluna

    def ultimo_para_primeiro(self):
        """
        Cria o mapeamento da última para a primeira coluna da BWT.

        Este método gera uma lista de índices correspondentes na primeira coluna (ordenada)
        para cada posição na BWT.

        Returns:
            list: Lista com os índices correspondentes na primeira coluna para cada caractere da BWT.
        """
        resultado = []                                                                                
        primeira_coluna = self.obtem_primeira_coluna()                                                 
        for i in range(len(primeira_coluna)):
            c = self.bwt[i]                                                                            
            ocorrencias = self.bwt[:i].count(c) + 1                                                    

            resultado.append(self.encontra_iesima_ocorrencia(primeira_coluna, c, ocorrencias))         
        return resultado

    def correspondencia_bw(self, padrao):
        """
        Realiza a procura backward search para encontrar o padrão na BWT.

        Utiliza o mapeamento 'último para o primeiro' para efetuar a procura do padrão 
        de forma eficiente na BWT.

        Args:
            padrao (str): O padrão a ser procurado.

        Returns:
            list: Lista de índices da BWT onde o padrão é encontrado.
        """
        lf = self.ultimo_para_primeiro()                                           
        resultado = []                                                              
        topo = 0                                                                    
        fundo = len(self.bwt) - 1                                                   
        continuar = True                                                            
        while continuar and topo <= fundo:
            if padrao != "":
                simbolo = padrao[-1]                                                
                padrao = padrao[:-1]                                                
                sublista = self.bwt[topo:(fundo + 1)]                               
                if simbolo in sublista:
                    indice_topo = sublista.index(simbolo) + topo                    
                    indice_fundo = fundo - sublista[::-1].index(simbolo)            
                    topo = lf[indice_topo]
                    fundo = lf[indice_fundo]
                else:
                    continuar = False                                              
            else:
                for i in range(topo, fundo + 1):                                    
                    resultado.append(i)
                continuar = False
        return resultado

    def correspondencia_bw_prefixo(self, padrao):
        """
        Realiza a correspondência backward search e retorna as posições relativas no texto original.

        O método utiliza o array de sufixos (sa) para converter os índices da BWT nas posições
        correspondentes do texto original.

        Args:
            padrao (str): O padrão a ser procurado.

        Returns:
            list: Lista ordenada com as posições correspondentes no texto original.
        """
        resultado = []                                                 
        correspondencias = self.correspondencia_bw(padrao)             
        for m in correspondencias:
            resultado.append(self.sa[m])                               
        resultado.sort()                                               
        return resultado

    def encontra_iesima_ocorrencia(self, lista, elemento, indice):
        """
        Encontra a i-ésima ocorrência de um elemento numa lista.

        Percorre a lista procurando pelo elemento e retorna a posição da ocorrência
        especificada.

        Args:
            lista (list): Lista onde o elemento será procurado.
            elemento: Elemento a procurar.
            indice (int): Número da ocorrência desejada (1ª, 2ª, etc.).

        Returns:
            int: O índice na lista da i-ésima ocorrência ou -1 se não for encontrada.
        """
        j, contagem = 0, 0                                         
        while contagem < indice and j < len(lista):
            if lista[j] == elemento:
                contagem += 1                                      
                if contagem == indice:
                    return j                                      
            j += 1                                                 
        return -1                                                 

if __name__ == "__main__":
    seq_dna = "ACGTACGT$"
    bwt_dna = BWT(seq_dna)
    print(f"\nSequência original: {seq_dna}")
    print(f"BWT: {bwt_dna.bwt}")
    
    original = bwt_dna.inversa_bwt()
    print(f"BWT inversa: {original}")
    print(f"Coincide com o original: {original == seq_dna}")


Sequência original: ACGTACGT$
BWT: TT$AACCGG
BWT inversa: ACGTACGT$
Coincide com o original: True
